In [ ]:
import os
import numpy as np
import librosa
import sounddevice as sd
import moviepy as mp

from slowdowner.audio import *

In [10]:
# Parameters
input_file = "./data/Moments_notice_rosenwinkel.mp3"         # or "input.wav" or "input.mp3"
start_time = 20.0                 # in seconds
end_time = 25.0                  # in seconds
slowdown_factor = 2.0           # 2x slower

In [5]:
# Step 1: Load audio trace
if input_file.lower().endswith((".mp4", ".mov", ".avi", ".mkv")):
    audio, sr = extract_audio_from_video(input_file, save_flag=True, output_path="./data/Moments_notice_rosenwinkel.mp3")
else:
    audio, sr = load_audio(input_file)

Loading audio from ./data/Moments_notice_rosenwinkel.mp3...


In [6]:
# Step 2: Extract time window
audio_segment = extract_time_window(audio, sr, start_time, end_time)

In [11]:
# Step 3: Slow down the segment
slowed_segment = slow_down_audio(audio_segment, slowdown_factor)

Slowing down audio by a factor of 2.0 (without pitch change)...


In [12]:
# Step 4: Play the slowed segment in loop
play_audio_loop(slowed_segment, sr)

Playing slowed audio in loop. Press Ctrl+C to stop.
